In [ ]:
#Import libraries
import os
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load data
df_train_label = pd.read_csv('/Users/toanngo/Documents/GitHub/capstone 2 data/raw data /train_labels.csv')
df_test = pd.read_csv('/Users/toanngo/Documents/GitHub/capstone 2 data/raw data /test_data.csv', nrows = 10**5)
df_train = pd.read_csv('/Users/toanngo/Documents/GitHub/capstone 2 data/raw data /train_data.csv', nrows = 10**5)

In [ ]:
#Look at the train data train_label
#Note that D = Delinquency, S = Spend, P = Payment, B = Balance, R = Risk
df_train_label.head()
df_train_label.info()
df_train_label.shape

In [ ]:
#Look at the test data
df_test.head()
df_test.info()
df_test.shape

In [ ]:
#Look at the train data
df_train.head()
df_train.info()
df_train.shape

In [ ]:
print(df_train.columns == df_test.columns)

The train data has same columns with the test data with different year on S_2. It seems that American express is trying to predict behavior of customer in 2019 from the behavior of customer from previous years. Moreover, because they have same columns, the data cleaning we apply to train data can be applied to test data as well.

In [ ]:
#check if customer_ID is unique for the data sets
df_train_label.customer_ID.is_unique
df_test.customer_ID.is_unique
df_train.customer_ID.is_unique

In [ ]:
#Since the customer_ID is non-unique for test and train data, we figure out what is the issue
df_test.customer_ID.value_counts()
df_train.customer_ID.value_counts()

In [ ]:
#We investigate the first customer_ID from train set'0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'
df_train[df_train.customer_ID == '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a']

This customer had 13 transactions from 2017 to 2018 which might contain important info for fraud detection. Therefore, we can not drop the duplicate customer_ID for train and test table. We now investigate further train and test table.

In [ ]:
#Investigate df_train and df_test
categorical_train = df_train.select_dtypes(include=['object']) #select categorical columns
categorical_test = df_test.select_dtypes(include=['object'])

In [ ]:
df_train.D_63.value_counts()
df_train.D_64.value_counts()
df_test.D_63.value_counts()
df_test.D_64.value_counts()

Since there are only 2 categorical columns and they have limited distinct values, we can consider using one hot encoding when we build our model.

In [ ]:
#Check missing value, note that these tables have 100000 rows each
missing_train = df_train.isna().sum().sort_values(ascending = False).to_frame('missing_count')
missing_test = df_test.isna().sum().sort_values(ascending = False).to_frame('missing_count')

In [ ]:
#create a list of columns we will drop
to_drop_train = missing_train[missing_train['missing_count'] > 95000] #we drop if 95% or more is NaN
to_drop_test = missing_test[missing_test['missing_count'] > 95000]

In [ ]:
print(to_drop_train)
print(to_drop_test)

Although there is some small difference in the order, the lists of dropped columns are the same for train and test data.

In [ ]:
#drop unneccessary columns
to_drop = ['D_87', 'D_88', 'D_108', 'D_111', 'D_110', 'B_39', 'D_73', 'B_42', 'D_134', 'D_138', 'D_135', 'D_136', 'D_137']
df_train = df_train.drop(to_drop, axis = 1)
df_test = df_test.drop(to_drop, axis = 1)

In [ ]:
#plot for total missing values of columns
plt.figure(figsize = (20,5))
plt.plot(missing_train)
plt.show()

In [ ]:
#fill NaN value with 0
df_train.fillna(0, inplace = True)
df_test.fillna(0, inplace = True)

In [ ]:
#join with label, note that we only have label for train data set so we join df_train with label only
df_train = df_train.set_index('customer_ID').join(df_train_label.set_index('customer_ID'), on = 'customer_ID', how = 'inner')

In [ ]:
#save for later use
df_train.to_csv(path_or_buf='~/Documents/GitHub/capstone 2 data/processed data /df_train.csv', index=False)
df_test.to_csv(df_train.to_csv(path_or_buf='~/Documents/GitHub/capstone 2 data/processed data /df_test.csv', index=False)
, index=False)